In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [2]:
poker = pd.read_csv("poker.csv")
poker.columns.values
poker = poker.head(200000)

In [3]:
i = 100
j = 0
features = ['S1', ' C1', ' S2', ' C2', ' S3', ' C3', ' S4', ' C4', ' S5', ' C5']
for index, m in poker.iterrows():
    if index % 20 == 0:
        poker.at[index,features[j]] = i + 1
        j += 3
        i+=10
        if j >= 9:
            j = 0

In [5]:
i = 0
j = 0
for index, m in poker.iterrows():
    if index % 20 == 0:
        if index < 299994:
            poker.at[index+5,:] = poker.loc[j,:]
        else:
            poker.at[index+1,:] = poker.loc[j,:]
        j += 120
        if j >= 299994:
            j = 0

In [6]:
poker.to_csv('/home/valia/Documents/AppliedDataScience/pokerNoise.csv', sep=',')

In [7]:
poker

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,Class
0,101,12,3,2,3,11,4,5,2,5,1
1,1,9,4,6,1,4,3,2,3,9,1
2,1,4,3,13,2,13,2,1,3,6,1
3,3,10,2,7,1,2,2,11,4,9,0
4,1,3,4,5,3,4,1,12,4,6,0
5,101,12,3,2,3,11,4,5,2,5,1
6,3,2,4,9,3,7,4,3,4,5,0
7,4,4,3,13,1,8,3,9,3,10,0
8,1,9,3,8,4,4,1,7,3,5,0
9,4,7,3,12,1,13,1,9,2,6,0


In [8]:
data = poker.iloc[:,0:8]
labels = poker.iloc[:,8].apply(str)

In [9]:
kf = KFold(n_splits=2, shuffle=True) #5 fores me 2 folds

### Euclidean and k tuning on 10% noise dataset

In [10]:
acc = []
mean_accuracy_model_euclidean = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=1, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.30      0.29      0.29     37573
           3       0.30      0.29      0.29     37637
           2       0.29      0.29      0.29     37394
           4       0.29      0.30      0.30     37396

   micro avg       0.29      0.29      0.29    150000
   macro avg       0.29      0.29      0.29    150000
weighted avg       0.29      0.29      0.29    150000

accuracy:  0.2939733333333333
              precision    recall  f1-score   support

           1       0.29      0.30      0.30     37456
           3       0.29      0.29      0.29     37377
           2       0.29      0.29      0.29     37404
           4       0.29      0.29      0.29     37763

   micro avg       0.29      0.29      0.29    150000
   macro avg       0.29      0.29      0.29    150000
weighted avg       0.29      0.29      0.29    150000

accuracy:  0.2937666666666667
              precision    recall  f1-score   support

           1   

In [11]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=5, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.28      0.37      0.32     37486
           3       0.28      0.31      0.30     37505
           2       0.28      0.25      0.27     37273
           4       0.30      0.20      0.24     37736

   micro avg       0.28      0.28      0.28    150000
   macro avg       0.28      0.28      0.28    150000
weighted avg       0.29      0.28      0.28    150000

accuracy:  0.28336
              precision    recall  f1-score   support

           1       0.28      0.37      0.32     37543
           3       0.28      0.31      0.29     37509
           2       0.29      0.25      0.27     37525
           4       0.29      0.20      0.24     37423

   micro avg       0.28      0.28      0.28    150000
   macro avg       0.29      0.28      0.28    150000
weighted avg       0.29      0.28      0.28    150000

accuracy:  0.28396666666666665
              precision    recall  f1-score   support

           1       0.28  

In [12]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=10, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.36      0.30     37447
           3       0.27      0.28      0.27     37647
           2       0.27      0.23      0.25     37304
           4       0.27      0.20      0.23     37602

   micro avg       0.27      0.27      0.27    150000
   macro avg       0.27      0.27      0.26    150000
weighted avg       0.27      0.27      0.26    150000

accuracy:  0.26802
              precision    recall  f1-score   support

           1       0.26      0.35      0.30     37582
           3       0.26      0.28      0.27     37367
           2       0.27      0.23      0.25     37494
           4       0.27      0.20      0.23     37557

   micro avg       0.27      0.27      0.27    150000
   macro avg       0.27      0.27      0.26    150000
weighted avg       0.27      0.27      0.26    150000

accuracy:  0.2654666666666667
              precision    recall  f1-score   support

           1       0.26   

In [13]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=15, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.34      0.30     37420
           3       0.26      0.27      0.26     37498
           2       0.27      0.23      0.25     37393
           4       0.26      0.22      0.24     37689

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.26413333333333333
              precision    recall  f1-score   support

           1       0.26      0.33      0.29     37609
           3       0.27      0.27      0.27     37516
           2       0.27      0.24      0.25     37405
           4       0.26      0.22      0.24     37470

   micro avg       0.27      0.27      0.27    150000
   macro avg       0.27      0.27      0.26    150000
weighted avg       0.27      0.27      0.26    150000

accuracy:  0.26544666666666666
              precision    recall  f1-score   support

           1 

In [14]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=20, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.32      0.29     37454
           3       0.26      0.27      0.27     37378
           2       0.26      0.24      0.25     37516
           4       0.27      0.23      0.24     37652

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.26306
              precision    recall  f1-score   support

           1       0.26      0.32      0.29     37575
           3       0.26      0.26      0.26     37636
           2       0.26      0.24      0.25     37282
           4       0.26      0.23      0.25     37507

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.26378666666666667
              precision    recall  f1-score   support

           1       0.26  

In [15]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=30, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.32      0.29     37468
           3       0.26      0.26      0.26     37484
           2       0.26      0.23      0.25     37495
           4       0.27      0.23      0.25     37553

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.26271333333333335
              precision    recall  f1-score   support

           1       0.26      0.31      0.28     37561
           3       0.26      0.26      0.26     37530
           2       0.26      0.24      0.25     37303
           4       0.26      0.23      0.25     37606

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.26081333333333334
              precision    recall  f1-score   support

           1 

In [16]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=50, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.31      0.28     37380
           3       0.26      0.25      0.25     37511
           2       0.26      0.23      0.24     37444
           4       0.26      0.24      0.25     37665

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.25732666666666665
              precision    recall  f1-score   support

           1       0.26      0.30      0.28     37649
           3       0.26      0.26      0.26     37503
           2       0.26      0.23      0.24     37354
           4       0.26      0.24      0.25     37494

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.25812
              precision    recall  f1-score   support

           1       0.26  

In [17]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=100, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.29      0.27     37620
           3       0.26      0.25      0.25     37462
           2       0.26      0.23      0.24     37339
           4       0.25      0.26      0.26     37579

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.25675333333333333
              precision    recall  f1-score   support

           1       0.26      0.30      0.28     37409
           3       0.26      0.24      0.25     37552
           2       0.25      0.23      0.24     37459
           4       0.26      0.25      0.26     37580

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.2560866666666667
              precision    recall  f1-score   support

           1  

In [18]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=150, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

              precision    recall  f1-score   support

           1       0.26      0.30      0.28     37414
           3       0.25      0.24      0.25     37495
           2       0.25      0.23      0.24     37481
           4       0.26      0.26      0.26     37610

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.25    150000
weighted avg       0.26      0.26      0.25    150000

accuracy:  0.25549333333333335
              precision    recall  f1-score   support

           1       0.26      0.29      0.27     37615
           3       0.26      0.23      0.25     37519
           2       0.25      0.23      0.24     37317
           4       0.25      0.27      0.26     37549

   micro avg       0.26      0.26      0.26    150000
   macro avg       0.26      0.26      0.26    150000
weighted avg       0.26      0.26      0.26    150000

accuracy:  0.2562
              precision    recall  f1-score   support

           1       0.26   

In [19]:
print mean_accuracy_model_euclidean
k = [1, 5, 10, 15, 20, 30, 50, 100, 150]
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_euclidean)
plt.show()

[0.293926, 0.2834966666666666, 0.2669606666666667, 0.2652326666666667, 0.26354733333333336, 0.26250133333333336, 0.25852533333333333, 0.25624399999999997, 0.25578333333333336]


NameError: name 'plt' is not defined

### Minkowski and k tuning on 10% dataset

In [ ]:
acc = []
mean_accuracy_model_minkowski = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=1, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=5, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=10, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=15, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=20, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=30, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=50, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=100, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=150, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
print mean_accuracy_model_minkowski
k = [1, 5, 10, 15, 20, 30, 50, 100, 150]
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_minkowski)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_minkowski)
ax.plot(k, mean_accuracy_model_euclidean)
plt.show()